# Description

This demo notebook shows how to create a Keras model for scoring. It's based upon the tutorial here: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/. It requires that you first download the data. Next, you can see how to create the model and use it for scoring. It shows how to create the model all in memory, or using the file system. Lastly, it requests has you create storage (both ADLS and blob) to store the model and data for the subsequent sample notebooks. 

# Download data

Assuming you are using a Windows DSVM, by default Internet Explorer doesn't have permissions set to allow file
downloads. To fix this:
    
1. Open Internet Explorer.
1. Click Tools in the Menu bar (upper-right)
1. Select Internet options from the drop down list.
1. Select the Security TAB and select Internet from the Select the zone to view or change security settings.
1. Now click on Custom level, new windows will appear where you have to scroll down and search for the option File Download and you have to select the radio button Enable and then click on Ok.

Next download the data.

1. Browse to here: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
1. Scroll down to the Load Data section.
1. Right click the Dataset File to download the data to a local directory (e.g. c:\keras).

NOTE: You can learn more about the data by clicking the Dataset Details link.

In [ ]:
# Load Data

from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)

# load pima indians dataset
# https://stackoverflow.com/questions/47039720/python-numpys-loadtxt-and-windows-pathing
dataset = numpy.loadtxt("C:/keras/pima-indians-diabetes.data.txt", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [ ]:
X

In [ ]:
Y

In [ ]:
# Define Model

model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile Model

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit Model

model.fit(X, Y, epochs=150, batch_size=10)

In [ ]:
# Evaluate Model

# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# Make Predictions

# calculate predictions
predictions = model.predict(X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

In [ ]:
# Export Model

from keras.models import load_model

# serialize model to JSON
model_json = model.to_json()
with open("C:/keras/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("C:/keras/weights.h5")
print("Saved model to disk")

In [ ]:
# Import Model

from keras.models import model_from_json

# https://machinelearningmastery.com/save-load-keras-deep-learning-models/


# load json and create model
json_file = open('C:/keras/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("C:/keras/weights.h5")
print("Loaded model from disk")

In [ ]:
# Make Predictions

# calculate predictions
predictions = loaded_model.predict(X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

# Upload model

Create an Azure Data Lake Storage (ADLS) resource, create the folders 'input', 'model', and 'output'.

- Upload the data to the ADLS 'input' folder
- Upload the model files, .json and .h5, to the ADLS 'model' folder

Create a Storage Account, and create a container called 'batchscoring'